<a href="https://colab.research.google.com/github/juno2856/deeplearning/blob/master/NLP/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_20220519.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Bag of words(Bow)

단어의 등장 순서를 고려하지 않은 빈도수 기반의 단어 표현 방법
1. 각 단어의 고유한 정수 인덱스를 부여
2. 각 인덱스 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만든다.

doc1 = 'John likes to watch movies. Mary likes movies too'  
Bow1 = {"Jonh" : 1, "likes" :2, "to" : 1, "watch" :1, "movies" : 2, "Mary" : 1, "too" : 1}

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 48.1 MB/s 


In [ ]:
from konlpy.tag import Okt
import re

okt = Okt()

# 정규표현식을 통해 온점을 제거하는 정제 작업
token = re.sub("(\.)", "", "소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.")

token = okt.morphs(token) # okt 형태소분석기

In [ ]:
token

['소비자', '는', '주로', '소비', '하는', '상품', '을', '기준', '으로', '물가상승률', '을', '느낀다']

In [ ]:
word2index = {}
bow = []

for voca in token:
    if voca not in word2index.keys():
        # token을 읽으면서, word2index에 없는 단어는 새로 추가하고, 이미 있는 단어는 넘긴다.
        word2index[voca] = len(word2index) # {'소비자' : 0 }
        # bow 전체에 전부 기본값을 1을 넣어준다. 단어 갯수는 최소 1개 이상이기 때문에
        bow.insert(len(word2index)-1, 1)
    else:
        index = word2index.get(voca)
        # 재 등장하는 단어의 인덱스를 받아오기
        bow[index] = bow[index]+1
        # 재 등장하는 단어의 해당하는 인덱스의 위치에 1을 더해줌. (단어 갯수를 세는 것)

print(word2index) # 단어장

{'소비자': 0, '는': 1, '주로': 2, '소비': 3, '하는': 4, '상품': 5, '을': 6, '기준': 7, '으로': 8, '물가상승률': 9, '느낀다': 10}


In [ ]:
bow

[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]

# Tensorflow의 keras Tokenizer를 활용한 BOW

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
sentence = ["John likes to watch movies. \
            Mary likes movies too! \
            Mary also likes to watch football games."]

In [ ]:
def print_bow(sentence):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(sentence)

  bow = dict(tokenizer.word_counts) # 각 단어와 각단어의 빈도를 bow 저장

  print('Bag of words : ', bow)
  print('단어장(vocabulary)의 크기 : ', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 갯수

In [ ]:
print_bow(sentence)

Bag of words :  {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(vocabulary)의 크기 :  10


# scikit-learn의 CountVectorizer을 활용한 Bow

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
sentence = ["John likes to watch movies. \
            Mary likes movies too! \
            Mary also likes to watch football games."]

In [ ]:
vector = CountVectorizer()

In [ ]:
print("Bag of words : ", vector.fit_transform(sentence).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록
print("각 단어의 인덱스 :", vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되는지를 보여줌

Bag of words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


# 불용어를 제거한 BOW만들기

사용자가 직접 정의한불용어 사용



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text = ["Family is not an important thing. It's everything"]

vect = CountVectorizer(stop_words = ["the", "a", "an", "is", "not"])

print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


# CountVectorizer에서 제공하는 자체 불용어 사용

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

text = ["Family is not an important thing. It's everything"]

vect = CountVectorizer(stop_words = 'english')

print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


#NLTK에서 지원하는 불용어 사용
http://konltk.org/en/latest/

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

text = ["Family is not an important thing. It's everything"]

sw = stopwords.words('english') #nltk안에 자체 정의한 불용어

vect = CountVectorizer(stop_words= sw)

print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


# DTM(Document-Term-Matrix

In [ ]:
import pandas as pd
content = [[0,1,1,1],[1,0,1,1],[2,0,2,2]]
df = pd.DataFrame(content)
df.index = ['(문서1) I like dog', '(문서2) I like cat', '(문서3) I like cat I like cat']
df.columns = ['cat', 'dog', 'I', 'like']
df

,cat,dog,I,like
(문서1) I like dog,0,1,1,1
(문서2) I like cat,1,0,1,1
(문서3) I like cat I like cat,2,0,2,2


In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

In [ ]:
def cos_sim(a,b):
  return dot(a,b)/(norm(a * norm(b)))
# 코사인 유사도는 0-1 사이의 값을 가지고 1에 가까울 수록 유사도 가 높다고 판단

In [ ]:
print(cos_sim(doc1, doc2))
print(cos_sim(doc1, doc3))
print(cos_sim(doc2, doc3))

0.6666666666666667
0.6666666666666667
1.0000000000000002


# Scikit_learn CountVectorizer를 활용한 DTM 구현

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ['John likes to watch movies.',
          'Mary likes movies too!',
          "Mary also likes to watch football games."]

vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


# TF-IDF(Term_Frequency-Inverse DocumentFrequency)

모든 문서에 자주 등장하는 단어는 중요도가 낮다고 판단하고, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단하는것

In [ ]:
from math import log
import pandas as pd

docs = ['John likes to watch movies and Mary likes movies too',
        'James likes to watch TV',
        'Mary also likes to watch football games']

In [ ]:
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [ ]:
print('단어장의 크기 : ', len(vocab))
print(vocab)

단어장의 크기 :  13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [ ]:
N = len(docs)
N

3

1. tf(t,d) : 특정 문서 d에서의 특정 단어 t의 등장 횟수
2. df(t) : 특정 단어 t가 등장한 문서의 수
3. idf(t,d) : df(t)에 반비례하는 수

$$ idf(d, t) = log\frac{n}{1+df(t)}$$

In [ ]:
def tf(t, d): # 특정 문서 d에서의 특정 단어 t의 등장 횟수
  return d.count(t)

In [ ]:
def idf(t): # 특정 단어 t가 등장한 문서의 수
  df = 0
  for doc in docs:
    df+= t in doc
  return log(N/(df+1))+1

def tfidf(t,d):
  return tf(t,d) * idf(t)

In [ ]:
result = []
for i in range(N):
  result.append([])
  d = docs[i] # 문서
  for j in range(len(vocab)):
    t = vocab[j]

    result[-1].append(tf(t,d))

AttributeError: ignored

In [ ]:
tf_ = pd.DataFrame(result, columns = vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


In [ ]:
result = []
for j in range(len(vocab)):
  t = vocab[j]
  result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns = ['IDF'])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


In [ ]:
result = []
for i in range(N):
  result.append([])
  d= docs[i]
  for j in range(len(vocab)):
    t = vocab[j]

    result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_ 

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


# scikit_learn을 활용한 TF-IDF 구현

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
corpus = [
          'you know I want your love',
          'I like you',
          'what should I do'
]

In [ ]:
vector = CountVectorizer()

In [ ]:
print(vector.fit_transform(corpus).toarray())

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]


In [ ]:
print(vector.vocabulary_)

{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidfv =  TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]


In [ ]:
print(tfidfv.vocabulary_)

{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
